# Demo of standard degridder

[Colab Link](https://colab.research.google.com/github/casangi/astroviper/blob/main/docs/core_tutorials/imaging/demo_standard_degrid.ipynb)


Description of notebook

The intial demo is to Fourier transform an image with 4 sources.
We degrid that onto the VISIBILITY_MODEL of a xarray ms4 dataset.
Then we grid those model visibility data with the same convolution using the standard gridder and Fourier transform that back to an image and compare the source position and values
The second part compare with casa prediction using a standard gridder and a given model image.

## Assumptions

The centre pixel is assumed to be the phasecentre.

---
## Pseudo Code 

**For example:**

- ms4 Measurement Set v4 Data Tree
- grid is FT of model image
- pixelincr numparray of size 2 (dx, dy) with values in radians 
```
core/imaging/imaging_utils/standard_degrid.py

    degrid_spheroid_ms4(ms4, grid, pixelincr, incremental=False) 

    
```

The model visibilities is saved on the VISIBILITY_MODEL dataArray of ms4

---
## API

## Install AstroVIPER



In [ ]:
from importlib.metadata import version
import os

try:
    os.system("pip install --upgrade astroviper")

    import astroviper

    print("Using astroviper version", version("astroviper"))

except ImportError as exc:
    print(f"Could not import astroviper: {exc}")

In [5]:
from astroviper.core.imaging.imaging_utils.standard_degrid import degrid_spheroid_ms4
help(degrid_spehroid_ms4)    


NameError: name 'degrid_spehroid_ms4' is not defined

## degridding followed by gridding

We 